In [1]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from lagged_coherence import *
import colorednoise as cn
import scipy

In [2]:
def compute_rms(time_series):
    return np.sqrt(np.mean(np.square(time_series)))

def scale_noise(signal_without_noise, noise, desired_snr_db):
    # Compute power of the original signal
    Ps = compute_rms(signal_without_noise)**2
    
    # Convert desired SNR from dB to linear scale
    snr_linear = 10**(desired_snr_db / 10)
    
    # Calculate the desired noise power based on the desired SNR
    desired_noise_power = Ps / snr_linear
    
    # Compute scaling factor for the noise
    alpha = np.sqrt(desired_noise_power / compute_rms(noise)**2)
    
    return alpha * noise

def gen_signal(T, trials, srate, fs, snr_db):
    time=np.linspace(0,T,T*srate)
    
    w = 2. * np.pi * f
    signal=np.zeros((trials,len(time)))
    for i in range(trials):
        pure_signal = np.sin(w * (time + np.random.randn()))
        noise = cn.powerlaw_psd_gaussian(1, len(time))
        scaled_noise = scale_noise(pure_signal, noise, snr_db)        
        
        signal[i,:]=pure_signal+scaled_noise
        
    return signal

In [3]:
T=5
trials=100
srate=1000
snr=0

lags=np.arange(1,6.5,.5)
osc_f=np.arange(10,55,5)

f_min=5
f_max=100
n_freqs=((f_max-f_min)*2)+1
lcs_classic=np.zeros((len(osc_f), trials, n_freqs, len(lags)))
lcs_hilbert=np.zeros((len(osc_f), trials, n_freqs, len(lags)))
psds=np.zeros((len(osc_f), trials, n_freqs))

for f_idx, f in enumerate(osc_f):
    print('{}Hz'.format(f))
    signal=gen_signal(T, trials, srate, f, snr)

    freqs, psd = scipy.signal.welch(signal, fs=srate, window='hann',
                                    nperseg=srate, noverlap=int(srate / 2), nfft=srate * 2, detrend='constant',
                                    return_onesided=True, scaling='density', axis=- 1, average='mean')
    idx=(freqs>=f_min) & (freqs<=f_max)
    freqs=freqs[idx]
    psd=psd[:,idx]
    psds[f_idx,:,:]=psd

    lcs_classic[f_idx,:,:,:]=lagged_coherence(signal, freqs, lags, srate)
    
    lcs_hilbert[f_idx,:,:,:]=lagged_hilbert_coherence(signal, freqs, lags, srate, n_jobs=20)
    np.savez('../output/sims/oscillation/sim_results', 
             freqs=freqs,
             lcs_classic=lcs_classic,
             lcs_hilbert=lcs_hilbert,
             psds=psds)


10Hz
15Hz
20Hz
25Hz
30Hz
35Hz
40Hz
45Hz
50Hz
